In [ ]:
!pip uninstall -y s3fs
!pip install s3fs==0.4.0

import s3fs
assert(s3fs.__version__  == "0.4.0")
bucket_name = "ucb-mids-wall-e-andy-test"

fs = s3fs.S3FileSystem(anon=False, key='', secret='')
print(fs.ls(bucket_name))

import sys
import urllib.request


In [ ]:
for fname in fs.ls(bucket_name + "/OpenStreetCam/openstreetcam_data_raw/"):
    with fs.open(fname, 'rb') as f_in:
        for raw_line in f_in:
            line = raw_line.decode("utf-8")
            split_line = line.split("\t")
            track_id, image_id, url = split_line[0], split_line[1], split_line[-1]
            target_name = bucket_name + "/OpenStreetCam/openstreetcam_data_images/{}_{}.jpg".format(track_id, image_id)
            print(target_name)

            if fs.exists(target_name):
                print("skipped")
                continue
                
            f = urllib.request.urlopen(url, timeout=1800)
            imbytes = f.read()
            
            if not imbytes is None:
                fs.touch(target_name)
                f_out = fs.open(target_name, "wb")
                f_out.write(imbytes)
                f_out.close()
                print("downloaded")
                
        f_in.close()
                
